# Modélisation géométrique d’un robot manipulateur

Le but de ce bureau d’études est de développer les modèles géométriques du robot industriel Universal Robot UR3. Dans un premier temps on calculera le MGD du 6R. Ensuite afin de simplifier le calcul du MGI on supposera que les 3 dernières liaisons sont fixes afin de se ramener à un robot 3R.

## Présentation du robot
L’UR3 d’Universal Robot est un robot 6R. La figure 1 vous indique ses axes de rotations ainsi que ses dimensions nécessaires pour établir les modèles.

La configuration du robot est définie dans l’espace généralisé par `q=(q1,q2,q3,q4,q5,q6)^T` .  
On ne considère pas de butée mécanique sur ce robot dans un premier temps, i.e., `q ∈ R6`.  
La situation de l’O.T. est définie par la position du point `Oe` et par l’orientation du repère de l’effecteur Re par rapport au repère de base `R0` , `(O0,x0,y0,z0)`.

![](pics/UR3.png)
![](pics/Dimensions.png)


## Modélisation du robot UR3

### Modélisation géométrique directe de l’UR3 (MGD)

Sachant que l’origine du repère 3 (O3) et que le repère outil (noté e) sont imposés, calculer le MGD de ce robot.

![](pics/without_repere.png)

#### Préparation
* En vous servant des données techniques (figure 1) indiquer les dimensions manquantes sur la figure 2.
* Placer les repères.
* Déterminer les paramètres de DHM associés au placement des repères (figure 2).

#### Travail à faire
* Utiliser les fonctions python du fichier pour calculer votre MGD de manière numérique.

In [8]:
import numpy as np
import math  as m

class UR3:
    def __init__(self):
        self.nb_ddl =   6
        self.o0o1   =   152    # mm
        self.o1o2   =   120    # mm
        self.o2o3   =   244    # mm
        self.o3o4   =   np.sqrt((213**2)+((93-83)**2))    # mm
        self.o4o5   =   83     # mm
        self.o6oE   =   82     # mm
        a3=213
        r4=10

        #Paramètres de DHM
        self.a=[0,0,self.o2o3,a3,0,0]
        self.alpha=[0,m.pi/2,0,0,m.pi/2,m.pi/2]
        self.r=[self.o0o1,-self.o1o2,0,r4,self.o4o5,0]
        # self.theta = qi 

    def MGD(self,q):

        # Calcul des matrices de passages
        for i in range(self.nb_ddl-1):
            T=np.array([[np.cos(q[i])                       ,   -np.sin(q[i])                       ,   0                       ,   self.a[i]], 
                        [np.cos(self.alpha[i])*np.sin(q[i]) ,   np.cos(self.alpha[i])*np.cos(q[i])  ,   -np.sin(self.alpha[i])  ,   -self.r[i]*np.sin(self.alpha[i])],
                        [np.sin(self.alpha[i])*np.sin(q[i]) ,   np.sin(self.alpha[i])*np.cos(q[i])  ,   np.cos(self.alpha[i])   ,   self.r[i]*np.cos(self.alpha[i])],
                        [0                                  ,   0                                   ,   0                       ,   1]])
            if i!=0:
                T0N=T0N@T
            else:
                T0N=T
            #print("T",i,i+1,"\n",T0N,"\n")

        Vp  = T0N[:3,3:]
        print("T06")
        print ("Xp = ", Vp[0][0])
        print ("Yp = ", Vp[1][0])
        print ("Zp = ", Vp[2][0])

        # Détermination de la position du repère de l'OT 
        T0E = T0N @ np.array([[ 1   ,   0   ,   0   ,   0           ], 
                              [ 0   ,   1   ,   0   ,   0           ],
                              [ 0   ,   0   ,   1   ,   self.o6oE   ],
                              [ 0   ,   0   ,   0   ,   1           ]])

        Vp  = T0E[:3,3:]
        print("\nT0E")
        print ("Xp = ", Vp[0][0])
        print ("Yp = ", Vp[1][0])
        print ("Zp = ", Vp[2][0])

        # Détermination de l'orientation du repère de l’OT avec les angles d'Euler
        la  = (m.atan2(-T0E[1][2],T0E[2][2]))**2
        nu  = m.asin(T0E[0][2])
        v   = m.atan2(T0E[0][1],T0E[0][0])
        print("\nAngles Euler")
        print ("la = ", la)
        print ("nu = ", nu)
        print ("v = ", v)

robot = UR3()
qdeg=[m.pi/2,0,0,m.pi/2,-m.pi,0]
robot.MGD(qdeg)

T06
Xp =  -109.99999999999996
Yp =  540.0
Zp =  152.0

T0E
Xp =  -109.99999999999994
Yp =  622.0
Zp =  152.0

Angles Euler
la =  2.46740110027234
nu =  1.2246467991473532e-16
v =  -1.5707963267948968


#### Vérification des résultats obtenu pour le MGD

Pour vérifier simplement les résultas du MGD on peut positionner le robot dans ça configuration initiale et vérifier qu'on obtient les cotes données en entrée. Soit :

* X = -110
* Y = 540 mm
* Z = 152 mm

## Robot RRR (limitation à 3 liaison de UR3)

In [40]:
import numpy as np

class UR3:
    def __init__(self):
        self.nb_ddl =   4
        self.o0o1   =   152    # mm
        self.o1o2   =   120    # mm
        self.o2o3   =   244    # mm
        self.o3o4   =   np.sqrt((213**2)+((93-83)**2))    # mm
        self.o4o5   =   83     # mm
        self.o6oE   =   82     # mm
        a3=213
        r4=10

        #Paramètres de DHM
        self.a=[0,0,self.o2o3,a3,0,0]
        self.alpha=[0,np.pi/2,0,0,np.pi/2,np.pi/2]
        self.r=[self.o0o1,-self.o1o2,0,r4,self.o4o5,0]
        # self.theta = qi 

    def MGD(self,q):

        # Calcul des matrices de passages
        for i in range(self.nb_ddl-1):
            T=np.array([[np.cos(q[i])                       ,   -np.sin(q[i])                       ,   0                       ,   self.a[i]], 
                        [np.cos(self.alpha[i])*np.sin(q[i]) ,   np.cos(self.alpha[i])*np.cos(q[i])  ,   -np.sin(self.alpha[i])  ,   -self.r[i]*np.sin(self.alpha[i])],
                        [np.sin(self.alpha[i])*np.sin(q[i]) ,   np.sin(self.alpha[i])*np.cos(q[i])  ,   np.cos(self.alpha[i])   ,   self.r[i]*np.cos(self.alpha[i])],
                        [0                                  ,   0                                   ,   0                       ,   1]])
            if i!=0:
                T0N=T0N@T
            else:
                T0N=T
            #print("T",i,i+1,"\n",T0N,"\n")

        Vp  = T0N[:3,3:]
        X=Vp[0][0]
        Y=Vp[1][0]
        Z=Vp[2][0]
        print("T03")
        print ("Xp = ", Vp[0][0])
        print ("Yp = ", Vp[1][0])
        print ("Zp = ", Vp[2][0])

        # Détermination de la position du repère de l'OT 
        T0E = T0N @ np.array([[ 1   ,   0   ,   0   ,   0           ], 
                              [ 0   ,   1   ,   0   ,   0           ],
                              [ 0   ,   0   ,   1   ,   self.o6oE   ],
                              [ 0   ,   0   ,   0   ,   1           ]])

        Vp  = T0E[:3,3:]
        print("\nT0E")
        print ("Xp = ", Vp[0][0])
        print ("Yp = ", Vp[1][0])
        print ("Zp = ", Vp[2][0])

        # Détermination de l'orientation du repère de l’OT avec les angles d'Euler
        la  = (np.arctan2(-T0E[1][2],T0E[2][2]))**2
        nu  = np.arcsin(T0E[0][2])
        v   = np.arctan2(T0E[0][1],T0E[0][0])
        print("\nAngles Euler")
        print ("la = ", la)
        print ("nu = ", nu)
        print ("v = ", v)

        return X,Y,Z
    
    def MGI(self,X,Y,Z):
        a=213+18
        b=93

        #x=np.cos(q[1])*np.cos(q[2])*(a*np.cos(q[3])+self.o2o3)-np.cos(q[1])*np.sin(q[2])*a*np.sin(q[3])+b*np.sin(q[2])-np.sin(q[1])*self.o1o2
        #y=np.sin(q[1])*np.cos(q[2])*(a*np.cos(q[3])+self.o2o3)-np.sin(q[1])*np.sin(q[2])*a*np.sin(q[3])-b*np.cos(q[1])+self.o1o2*np.cos(q[1])
        #z=np.sin(q[2])*(a*np.cos(q[3])+self.o2o3)+np.cos(q[2])*a*np.sin(q[3])+self.o0o1

        z1=np.sqrt(X**(2)+Y**(2)-(b-self.o2o3)**(2))
        z2=Z-self.o0o1

        c3=(z1**(2)+z2**(2)-X**(2)-Y**(2))/(2*X*Y)
        
        q3=np.arctan2(np.sqrt(1-c3**(2)),c3)

        b1=X+Y*c3
        b2=Y*np.sin(q3)

        s2=(b1*z2-b2*z1)/(b1**(2)+b2**(2))
        c2=(b1*z1+b2*z2)/(b1**(2)+b2**(2))

        q2=np.arctan2(s2,c2)
        q1=np.arcsinh((Y*z1+X*(b-self.o1o2))/((b-self.o1o2)**(2)+z1**(2)))
        
        return [q1,q2,q3]
    

    def Jacobienne(self,q):
        return np.array([   [np.cos(q[i])                       ,   -np.sin(q[i])                       ,   0], 
                            [np.cos(self.alpha[i])*np.sin(q[i]) ,   np.cos(self.alpha[i])*np.cos(q[i])  ,   0],
                            [np.sin(self.alpha[i])*np.sin(q[i]) ,   np.sin(self.alpha[i])*np.cos(q[i])  ,   0]]),


robot = UR3()
qdeg=[np.pi/2,0,0]
X,Y,Z=robot.MGD(qdeg)
q1,q2,q3=robot.MGI(X,Y,Z)
print("qdeg=",qdeg[0],qdeg[1],qdeg[2])
print("q=",q1,q2,q3)

T03
Xp =  -119.99999999999999
Yp =  244.0
Zp =  152.0

T0E
Xp =  -37.999999999999986
Yp =  244.0
Zp =  152.0

Angles Euler
la =  0.6168502750680849
nu =  1.5707963267948966
v =  -0.7853981633974483
qdeg= 1.5707963267948966 0 0
q= 0.9679411672545349 -1.6815598807021572 1.1708582149494051
